In [1]:
import numpy as np
import random
import time

In [2]:
#Modelling a Maze
def gen_maze(N):
    
    randRow = N
    randCol = N
    steps = N

    i = 0
    j = 0

    mazeMap = [[0 for x in range(randCol)] for y in range(randRow)]
    mazeMap[i][j] = 'S'

    while steps != 0:
        iORj = random.choice([True, False])
        incORdec = random.randint(0, 5)
        if iORj and (incORdec > 0) and i != randRow - 1 and mazeMap[i + 1][j] != 'S' and mazeMap[i + 1][j] != '.':
            i = i + 1
            mazeMap[i][j] = '.'
            if steps == 1:
                mazeMap[i][j] = 'E'
        elif iORj == False and (incORdec > 0) and j != randCol-1 and mazeMap[i][j+1] != 'S' and mazeMap[i][j + 1] != '.':
            j = j + 1
            mazeMap[i][j] = '.'
            if steps == 1:
                mazeMap[i][j] = 'E'
        elif iORj and (incORdec == 0) and i != 0 and mazeMap[i-1][j] != 'S' and mazeMap[i-1][j] != '.':
            i = i - 1
            mazeMap[i][j] = '.'
            if steps == 1:
                mazeMap[i][j] = 'E'
        elif iORj == False and (incORdec == 0) and j != 0 and mazeMap[i][j-1] != 'S' and mazeMap[i][j-1] != '.':
            j = j -1
            mazeMap[i][j] = '.'
            if steps == 1:
                mazeMap[i][j] = 'E'
        else:
            continue
        steps = steps - 1

    ii = 0
    jj = 0

    for ii in range(0, randRow):
        for jj in range(0, randCol):
            iORj = random.choice([True, False])
            if mazeMap[ii][jj] != 'S':
                if mazeMap[ii][jj] != '.':
                    if mazeMap[ii][jj] != 'E':
                        if iORj:
                            mazeMap[ii][jj] = '*'
                        else:
                            mazeMap[ii][jj] = '.'
    return mazeMap




In [3]:
def get_goal_indx(maze,N):
    
    s=0
    for i in range(N):
        for j in range(N):
            if maze[i][j] == 'E':
                return s
            s+=1
    return -1

In [4]:
def get_next_state(maze,N):
    
    indx = np.zeros([N,N])    
    s=0
    for i in range(N):
        for j in range(N):
            indx[i][j]=s
            s+=1    
    next_state = np.zeros([N*N, 4])
    s=0
    for i in range(N):
        for j in range(N):
            if(i-1<0 or maze[i-1][j] == '*'):
                next_state[s][0] = s
            else:
                next_state[s][0] = indx[i-1][j]
                
            if(j+1>N-1 or maze[i][j+1] == '*'):
                next_state[s][1] = s
            else:
                next_state[s][1] = indx[i][j+1]
                
            if(i+1>N-1 or maze[i+1][j] == '*'):
                next_state[s][2] = s
            else:
                next_state[s][2] = indx[i+1][j]
                
            if(j-1<0 or maze[i][j-1] == '*'):
                next_state[s][3] = s
            else:
                next_state[s][3] = indx[i][j-1]
            
            if(maze[i][j] == '*' or ((i-1<0 or maze[i-1][j] == '*') and (j+1>N-1 or maze[i][j+1] == '*') and (i+1>N-1 or maze[i+1][j] == '*') and (j-1<0 or maze[i][j-1] == '*'))):
                next_state[s][0] = -1
                next_state[s][1] = -1
                next_state[s][2] = -1
                next_state[s][3] = -1
                
            if(maze[i][j] == 'E'):
                next_state[s][0] = s
                next_state[s][1] = s
                next_state[s][2] = s
                next_state[s][3] = s
            s+=1
    return next_state

In [5]:
def policy_eval(policy, reward, next_state,V_old, discount_factor=1.0, theta=0.00001):
    
    V_new = np.zeros(16)    
    for s in range(16):
        v = 0.0
        for a, action_prob in list(enumerate(policy[s])):           
            nxt = next_state[s][a]
            if(nxt != -1):
                v += action_prob * (reward + discount_factor * V_old[int(nxt)])            
        V_new[s] = v      
    
    return np.array(V_new)

In [6]:
def best_action(a):
    
    if np.array_equal(a,[0,0,0,0]) or np.array_equal(a,[1,0,0,0])or np.array_equal(a,[0,1,0,0]) or np.array_equal(a,[0,0,1,0]) or np.array_equal(a,[0,0,0,1]):
        return np.argmax(a)
    freq = np.zeros(4)
    i=0
    while i<4: 
        j=0
        while j<4:
            if abs(a[i]-a[j]) < 0.00001:
                freq[i] += 1
            j+=1
        i+=1
    max_indx = np.argmax(a)
    if freq[max_indx]>2:
        return -1
    return np.argmax(a)

In [7]:
def is_deterministic(policy):
    
    rows = policy.shape[0]
    cols = policy.shape[1]
    for x in range(0, rows):
        for y in range(0, cols):
            if abs(policy[x,y]-0.25)<0.0001:
                return False
    return True 
    

In [8]:
def policy_improvement(reward, next_state, goal_indx, policy_eval_fn=policy_eval, discount_factor=1.0):
    

    def one_step_lookahead(state, V):
        
        A = np.zeros(4)
        i=0
        for a in range(4):
            nxt = next_state[state][a]
            if(nxt != -1):
                A[i] += (reward + discount_factor * V[int(nxt)])
            i = i+1
        return A
    
    policy = np.ones([16, 4]) / 4    
    policy[goal_indx] = np.zeros(4)
    initial_policy = policy.copy()
    
    V_old = np.zeros(16)
    V_new = np.zeros(16)
    
    k=0
    while True:  
        print ("Iteration ",k,":")
        policy[goal_indx] = np.zeros(4)
        policy_old = policy.copy()
        V_new = policy_eval_fn(initial_policy, reward, next_state, V_old)        
        V_old = V_new.copy()
        print("Cuurent Values:")
        print(V_new)
        policy_stable = True
        
        for s in range(16):
            
            action_values = one_step_lookahead(s, V_new)
            best_a = best_action(action_values)
            
            if (best_a == -1):
              policy_stable = False
            if (best_a != -1):

              policy[s] = np.eye(4)[best_a]
        
        k+=1
        print("Current Policy Probability distribution: ")
        print(policy)
        if np.array_equal(policy,policy_old) and k>1 and is_deterministic(policy):
            return (policy, V_new)

In [9]:
def value_iteration(nS,goal_indx,discount_factor = 1.0,theta = 0.0001):
    
    def one_step_lookahead(state, V):
        
        A = np.zeros(4)
        i=0
        for a in range(4):
            nxt = next_state[s][a]
            if(nxt != -1):
                A[i] += (reward + discount_factor * V[int(nxt)])
            i = i+1
        return A
    
    V_old = np.zeros(nS)
    V_new = np.zeros(nS)
    while True:
        delta = 0
        V_old = V_new.copy()
        for s in range(nS):
            if s == goal_index:
                continue
            A = one_step_lookahead(s, V_old)
            best_action_value = np.max(A)
            delta = max(delta, np.abs(best_action_value - V_old[s]))
            V_new[s] = best_action_value    
        print(V_new)
        if delta < theta:
            break
    
    npolicy = np.zeros([nS, 4])
    for s in range(nS):
        A = one_step_lookahead(s, V_new)
        best_action = np.argmax(A)
        npolicy[s, best_action] = 1.0
    
    return npolicy, V_new

In [10]:
def get_path_policy(p):
    finished = False
    path = []
    actions = []
    next_square = 0
    current_square = 0
    while finished == False:
        finished = True
        for i in range(4):
            if p[next_square][i] == 1:
                finished = False
                if i == 0:
                    next_square -= N
                    actions.append("up")
                elif i == 1:
                    next_square += 1 
                    actions.append("right")
                elif i == 2:
                    next_square += N 
                    actions.append("bottom")
                else:
                    next_square -= 1
                    actions.append("left")
                path.append(next_square)
    return (path,actions)

In [11]:
def getMaxValueState(next_states,V):
    max = -99999999999
    index = 0
    for ii in range(4):
        c_val = V[int(next_states[ii])]
        if c_val > max:
            max = c_val
            index = ii
    return ii

In [12]:
def get_path_value(V,next_state):
    finished = False
    path = []
    actions = []
    next_square = 0
    current_square = 0
    while finished == False:
        finished = True
        for i in range(4):
            new_next_square = getMaxValueState(next_state[next_square],V)
            if new_next_square != next_square:
                finished = False
                next_square = new_next_square
                path.append(next_square)
                if i == 0:
                    actions.append("up")
                elif i == 1:
                    actions.append("right")
                elif i == 2:
                    actions.append("bottom")
                else:
                    actions.append("left")
    return (path,actions)

In [13]:
N = 4
maze = gen_maze(N)
reward = -1
discount_factor = 1.0
goal_index = get_goal_indx(maze,N) 
next_state = list(get_next_state(maze,N))

In [14]:
for ii in range(N):
    print(maze[ii])

['S', '.', '.', '.']
['.', '.', '.', '.']
['.', '.', 'E', '*']
['*', '.', '.', '.']


In [15]:
#policy iteration
start_time = time.time()
policy, v = policy_improvement(reward, next_state , goal_index)
exec_time = (time.time() - start_time)
print("\n\n-------------Final Results---------\n\n-")
print("Policy Probability Distribution:")
print(policy)
print("")
print("Value Function:")
print(v)
print("")
path,actions = get_path_policy(policy)
print("Path: ")
print(path)
print("Actions: ")
print(actions)
print("--- Running Time : %s seconds ---" % exec_time)


Iteration  0 :
Cuurent Values:
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  0.  0.  0. -1. -1. -1.]
Current Policy Probability distribution: 
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   1.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]
Iteration  1 :
Cuurent Values:
[-2.   -2.   -2.   -2.   -2.   -2.   -1.75 -2.   -2.   -1.75  0.    0.
  0.   -2.   -1.75 -2.  ]
Current Policy Probability distribution: 
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   1.   0.   0.  ]
 [0.   0.   1.   0.  ]
 [0.   0.   0.   1.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [1.   0.   0.   0.  ]


In [16]:
#value iteration
start_time = time.time()
mpolicy, v = value_iteration(N*N,goal_index)
exec_time = (time.time() - start_time)
print("\n\n-------------Final Results---------\n\n-")
print("Value Function:")
print(v.reshape((N,N)))
print("")
path = []
actions = []
path,actions = get_path_policy(policy)
print("Path: ")
print(path)
print("Actions: ")
print(actions)
print("--- Running Time : %s seconds ---" % exec_time)


[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  0.  0.  0. -1. -1. -1.]
[-2. -2. -2. -2. -2. -2. -1. -2. -2. -1.  0.  0.  0. -2. -1. -2.]
[-3. -3. -2. -3. -3. -2. -1. -2. -2. -1.  0.  0.  0. -2. -1. -2.]
[-4. -3. -2. -3. -3. -2. -1. -2. -2. -1.  0.  0.  0. -2. -1. -2.]
[-4. -3. -2. -3. -3. -2. -1. -2. -2. -1.  0.  0.  0. -2. -1. -2.]


-------------Final Results---------

-
Value Function:
[[-4. -3. -2. -3.]
 [-3. -2. -1. -2.]
 [-2. -1.  0.  0.]
 [ 0. -2. -1. -2.]]

Path: 
[4, 5, 9, 10]
Actions: 
['bottom', 'right', 'bottom', 'right']
--- Running Time : 0.005521535873413086 seconds ---
